In [18]:
import pandas as pd
import numpy as np

In [19]:
path = 'data//'
n = 10  ## <-- This needs to be changed

In [27]:
## Importing dataset
df = pd.read_csv(path + 'grid_' + str(n) + '_nodes.csv')
# df

In [21]:
## H resistors

H = pd.DataFrame()
# dimentions are n rows and n-1 columns

def reset_H():    
    for i in range(1, n+1):
        for j in range(1, n):
            # print(j, i, df_nodes.loc[i, j], df_nodes.loc[j, i])
            if i == 1 or i == n:
                H.loc[i, j] = df_nodes.loc[i, j] - df_nodes.loc[i, j+1]
            else:
                H.loc[i, j] = 0
    # print(H)


In [23]:
## V resistors

V = pd.DataFrame() 
# dimentions are n-1 rows and n columns

def reset_V():    
    for i in range(1, n):
        for j in range(1, n+1):
            if j == 1 or j == n:
                V.loc[i, j] = df_nodes.loc[i, j] - df_nodes.loc[i+1, j]
            else:
                V.loc[i, j] = 0
    # print(V)


In [24]:
## Funtion ROWS() to go through rows and check the resistors

def ROWS():
    for m in range(1, n):
        ## STEP 1: Creating V resistors voltage differences
        
        for i in range(m, m+1): # till n
            for j in range(2, n):
                # print(i, j, df_columns.loc[i, j])
                if i == 1 :
                    V.loc[i, j] = H.loc[i, j-1] - H.loc[i, j]
                else:
                    V.loc[i, j] = H.loc[i, j-1] - H.loc[i, j] + V.loc[i-1, j]
                if 0 < V.loc[i, j] < 0.00001:
                    print("ERROR HERE ", i, j)
                    return 'V', i, j
        # print("\nV resistors:\n", V)
        
        ## STEP 2: Creating H resistors voltage differences
        
        for i in range(m+1, m+2):
            if i == n:
                break
            for j in range(1, n):
                H.loc[i, j] = -V.loc[i-1, j] + H.loc[i-1, j] + V.loc[i-1, j+1]
                if 0 < H.loc[i, j] < 0.00001:
                    print("ERROR HERE ", i, j)
                    return 'H', i, j
        # print("\nH resistors:\n", H)
   
        # print("\n\n\nNEXT Row ", m, "\n\n\n")
    return '', 0, 0

# print("H ", ROWS(), " is broken")

In [25]:
## Function COLUMNS() to go though columns and check the resistors

def COLUMNS():
    for m in range(1, n):
        ## STEP 1: Creating V resistors voltage differences
        
        for j in range(m, m+1): # till n
            for i in range(2, n):
                # print(i, j, df_columns.loc[i, j])
                if j == 1 :
                    H.loc[i, j] = V.loc[i-1, j] - V.loc[i, j]
                else:
                    H.loc[i, j] = V.loc[i-1, j] - V.loc[i, j] + H.loc[i, j-1]
                if 0 < H.loc[i, j] < 0.00001:
                    print("ERROR HERE ", i, j)
                    return 'H', i, j
        # print("\nH resistors:\n", H)
        
        ## STEP 2: Creating H resistors voltage differences
        
        for j in range(m+1, m+2):
            if j == n:
                break
            for i in range(1, n):
                V.loc[i, j] = -H.loc[i, j-1] + V.loc[i, j-1] + H.loc[i+1, j-1]
                if 0 < V.loc[i, j] < 0.00001:
                    print("ERROR HERE ", i, j)
                    return 'V', i, j
        # print("\nV resistors:\n", V)
            
        # print("\n\n\nNEXT Row ", m, "\n\n\n")
    return '', 0, 0

# print("V ", COLUMNS(), " is broken")

In [26]:
## Function border_checker() to go over the border and check if one of them is broken 

def border_checker():
    for i in range(1, n+1):
        for j in range(1, n):
            if i == 1 or i == n:
                if 0 < H.loc[i, j] < 0.00001:
                    return 'H', i, j
    for i in range(1, n):
        for j in range(1, n+1):
            if j == 1 or j == n:
                if 0 < V.loc[i, j] < 0.00001:
                    return 'V', i, j
    return '', 0, 0

In [ ]:
## Main code

broken_data = []

for a in range(n * (n - 1) * 2): #n * (n - 1) * 2 + 1
    print("\nNEW BROKEN RESISTOR ", a)
    df_broken_node = df[a:a+1]
    data = df_broken_node.transpose()[:(n-1)*4][a].tolist()

    # making node grid
    df_nodes = pd.DataFrame()
    k = 0
    for i in range(1, n+1):
        for j in range(1, n+1):
            if i == 1 or i == n or j == 1 or j == n:
                df_nodes.loc[i, j] = data[k]
                k = k + 1
            else:
                df_nodes.loc[i, j] = 0      
    # print(df_nodes)

    # reset H and V
    reset_H()
    reset_V()

    border = border_checker();
    if border == ('', 0, 0):
        pass
    else:
        if border[0] == 'H':
            broken_data.append([border[1], border[2]+0.5])
        else:
            broken_data.append([border[1]+0.5, border[2]])
        print("BROKEN RESISTOR at border", border)
        continue

    # ROWS code
    rows = ROWS()
    if rows == ('', 0, 0):
        pass
    else:
        if rows[0] == 'H':
            broken_data.append([rows[1], rows[2]+0.5])
        else:
            broken_data.append([rows[1]+0.5, rows[2]])
        print("H ", rows, " is broken")
    # ROWS()
    
    # reset H and V
    reset_H()
    reset_V()

    # COLUMS code
    columns = COLUMNS()
    if columns == ('', 0, 0):
        pass
    else:
        if columns[0] == 'H':
            broken_data.append([columns[1], columns[2]+0.5])
        else:
            broken_data.append([columns[1]+0.5, columns[2]])
        print("V ", columns, " is broken")
    # COLUMNS()
   


NEW BROKEN RESISTOR  0
BROKEN RESISTOR at border ('H', 1, 1)

NEW BROKEN RESISTOR  1
BROKEN RESISTOR at border ('V', 1, 1)

NEW BROKEN RESISTOR  2
BROKEN RESISTOR at border ('H', 1, 2)

NEW BROKEN RESISTOR  3
ERROR HERE  1 2
V  ('V', 1, 2)  is broken

NEW BROKEN RESISTOR  4
BROKEN RESISTOR at border ('H', 1, 3)

NEW BROKEN RESISTOR  5
ERROR HERE  1 3
V  ('V', 1, 3)  is broken

NEW BROKEN RESISTOR  6
BROKEN RESISTOR at border ('H', 1, 4)

NEW BROKEN RESISTOR  7
ERROR HERE  1 4
V  ('V', 1, 4)  is broken

NEW BROKEN RESISTOR  8
BROKEN RESISTOR at border ('H', 1, 5)

NEW BROKEN RESISTOR  9
ERROR HERE  1 5
V  ('V', 1, 5)  is broken

NEW BROKEN RESISTOR  10
BROKEN RESISTOR at border ('H', 1, 6)

NEW BROKEN RESISTOR  11
ERROR HERE  1 6
V  ('V', 1, 6)  is broken

NEW BROKEN RESISTOR  12
BROKEN RESISTOR at border ('H', 1, 7)

NEW BROKEN RESISTOR  13
ERROR HERE  1 7
V  ('V', 1, 7)  is broken

NEW BROKEN RESISTOR  14
BROKEN RESISTOR at border ('H', 1, 8)

NEW BROKEN RESISTOR  15
ERROR HERE  1 8


In [11]:
df_broken = pd.DataFrame()
df_broken.loc[:, 'X'] = df.loc[:, 'X']
df_broken.loc[:, 'Y'] = df.loc[:, 'Y']
df_broken

,X,Y
0,1.0,1.5
1,1.5,1.0
2,1.0,2.5
3,1.5,2.0
4,1.0,3.5
...,...,...
176,10.0,6.5
177,10.0,7.5
178,10.0,8.5
179,10.0,9.5


In [12]:
df_predicted = pd.DataFrame(data=broken_data, columns=['X', 'Y'])
df_predicted

,X,Y
0,1.0,1.5
1,1.5,1.0
2,1.0,2.5
3,1.5,2.0
4,1.0,3.5
...,...,...
175,10.0,5.5
176,10.0,6.5
177,10.0,7.5
178,10.0,8.5


In [13]:
for a in range(n * (n - 1) * 2): #n * (n - 1) * 2 + 1
    # print(df_broken.loc[a])
    if df_broken.loc[a, 'X'] == df_predicted.loc[a, 'X'] and df_broken.loc[a, 'Y'] == df_predicted.loc[a, 'Y']:
        pass
    else:
        print("ERROR in ", a)